In [ ]:
from vosk import Model, KaldiRecognizer
from pydub import AudioSegment
import json
import subprocess
from fpdf import FPDF
from tqdm import tqdm
import pandas as pd
import numpy as np
import os

In [ ]:
path_to_directory = 'PATH_TO_YOUR_DIRECTORY'

In [ ]:
#Function to create folders with the same name as the individual episodes to store the transcript and summary files as pdfs
def make_folders(title):
  out = []
  for f in tqdm(os.listdir(path_to_directory+title)):
    if 'mp3' in f:
      try:
        os.mkdir(path_to_directory + title + '/' + f.replace('.mp3', ''))
        out.append(f.replace('.mp3', ''))
      except:
        print(f)
  return out

In [ ]:
info_all_podcasts = pd.read_csv(path_to_directory+'podcast.csv')
titles = info_all_podcasts['Show'].unique().tolist()

In [ ]:
episode_folders = []
for x in titles:
  folders = make_folders(x)
  episode_folders.append(folders)

In [ ]:
FRAME_RATE = 16000
CHANNELS = 1

In [ ]:
#Speech Recognition model from vosk library
model = Model(model_name = 'vosk-model-en-us-0.42-gigaspeech')

In [ ]:
def voiceRecognition(model, file):
  rec = KaldiRecognizer(model, FRAME_RATE)
  rec.SetWords(True)

  mp3 = AudioSegment.from_mp3(file)
  mp3 = mp3.set_channels(CHANNELS).set_frame_rate(FRAME_RATE)

  step = 45000
  transcript = ""

  for i in tqdm(range(0, len(mp3), step)):
    segment = mp3[i : i+step]
    rec.AcceptWaveform(segment.raw_data)
    result = rec.Result()

    text = json.loads(result)['text']
    transcript += text

  return transcript

In [ ]:
for title in tqdm(titles):
  for f in tqdm(os.listdir(path_to_directory+title)):
    if 'mp3' in f:
      transcript = voiceRecognition(model, path_to_directory+title+'/'+f)
      pdf = FPDF()
      pdf.add_page()
      pdf.set_font('Arial', size = 12)
      pdf.multi_cell(0, 10, txt = transcript)
      pdf.output(path_to_directory+title+'/'+f.replace('.mp3','') + '/' + f.replace('.mp3', '.pdf'))